# PĘTLA TRENINGOWA

In [ ]:
#Defining the model

model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)


#Training settings

optimizer = optim.SGD(model.parameters(),lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.1)



checkpoint_dir = './checkpoint/'+args.tag+'/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)



print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

#Training
metrics = {'train_loss':[],  'val_loss':[]}
constant_metrics = {'min_val_epoch':-1, 'min_val_loss':9999999999999999}

def train(epoch):
    global metrics,loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1


    for cnt,batch in enumerate(loader_train):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        optimizer.zero_grad()
        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)



        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip_grad)


            optimizer.step()
            #Metrics
            loss_batch += loss.item()
            print('TRAIN:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['train_loss'].append(loss_batch/batch_count)




def vald(epoch):
    global metrics,loader_val,constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1

    for cnt,batch in enumerate(loader_val):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            #Metrics
            loss_batch += loss.item()
            print('VALD:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['val_loss'].append(loss_batch/batch_count)

    if  metrics['val_loss'][-1]< constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] =  metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(),checkpoint_dir+'val_best.pth')  # OK
        check_test_performance()

print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()


    print('*'*30)
    print('Epoch:',args.tag,":", epoch)
    for k,v in metrics.items():
        if len(v)>0:
            print(k,v[-1])


    print(constant_metrics)
    print('*'*30)

    with open(checkpoint_dir+'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir+'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)



Data and model loaded
Checkpoint dir: ./checkpoint/social-tag/
Training started ...
TRAIN: 	 Epoch: 0 	 Loss: 0.016207104548811913
TRAIN: 	 Epoch: 0 	 Loss: 0.015882850624620914
TRAIN: 	 Epoch: 0 	 Loss: 0.015689566421012085
TRAIN: 	 Epoch: 0 	 Loss: 0.015459113288670778
TRAIN: 	 Epoch: 0 	 Loss: 0.015237803757190704
TRAIN: 	 Epoch: 0 	 Loss: 0.015015211887657642
TRAIN: 	 Epoch: 0 	 Loss: 0.014812008743839604
TRAIN: 	 Epoch: 0 	 Loss: 0.014622995280660689
TRAIN: 	 Epoch: 0 	 Loss: 0.01444061566144228
TRAIN: 	 Epoch: 0 	 Loss: 0.014245782885700465
TRAIN: 	 Epoch: 0 	 Loss: 0.014046812768686901
TRAIN: 	 Epoch: 0 	 Loss: 0.013863949881245693
TRAIN: 	 Epoch: 0 	 Loss: 0.01367377618757578
TRAIN: 	 Epoch: 0 	 Loss: 0.01348486150215779
TRAIN: 	 Epoch: 0 	 Loss: 0.013300742457310359
TRAIN: 	 Epoch: 0 	 Loss: 0.013114072848111391
TRAIN: 	 Epoch: 0 	 Loss: 0.013066317591823366
VALD: 	 Epoch: 0 	 Loss: 0.010786443948745728
VALD: 	 Epoch: 0 	 Loss: 0.012944892980158329
VALD: 	 Epoch: 0 	 Loss: 0.0

100%|██████████| 947/947 [18:03<00:00,  1.14s/it]

Testing ....


ADE: 1.1845755289836106  FDE: 1.1428070892911715
**************************************************
******************************
Epoch: social-tag : 0
train_loss 0.013066317591823366
val_loss 0.01148518037121251
{'min_val_epoch': 0, 'min_val_loss': 0.01148518037121251}
******************************
TRAIN: 	 Epoch: 1 	 Loss: 0.009672910906374454
TRAIN: 	 Epoch: 1 	 Loss: 0.009489190764725208
TRAIN: 	 Epoch: 1 	 Loss: 0.009329176507890224
TRAIN: 	 Epoch: 1 	 Loss: 0.009113092673942447
TRAIN: 	 Epoch: 1 	 Loss: 0.008886980265378952
TRAIN: 	 Epoch: 1 	 Loss: 0.008760280596713224
TRAIN: 	 Epoch: 1 	 Loss: 0.008615195817713226
TRAIN: 	 Epoch: 1 	 Loss: 0.008419841702561826
TRAIN: 	 Epoch: 1 	 Loss: 0.008267498161229823
TRAIN: 	 Epoch: 1 	 Loss: 0.008108079293742776
TRAIN: 	 Epoch: 1 	 Loss: 0.007937206971374426
TRAIN: 	 Epoch: 1 	 Loss: 0.007883728404218951
TRAIN: 	 Epoch: 1 	 Loss: 0.007777526867217743
TRAIN: 	 Epoch: 1 	 Loss: 0.007643613631704024
TRAIN: 	 Epoch: 1 	 Loss: 0.00747766889

100%|██████████| 947/947 [18:02<00:00,  1.14s/it]

...

Testing ....


ADE: 0.5488509339056464  FDE: 0.70580472704967
**************************************************
******************************
Epoch: social-tag : 22
train_loss -0.0053065441466279345
val_loss -0.004679022841858414
{'min_val_epoch': 22, 'min_val_loss': -0.004679022841858414}
******************************
TRAIN: 	 Epoch: 23 	 Loss: -0.00690301600843668
TRAIN: 	 Epoch: 23 	 Loss: -0.00680499873124063
TRAIN: 	 Epoch: 23 	 Loss: -0.0068361032754182816
TRAIN: 	 Epoch: 23 	 Loss: -0.0069210107903927565
TRAIN: 	 Epoch: 23 	 Loss: -0.0066384849138557914
TRAIN: 	 Epoch: 23 	 Loss: -0.005852779819785307
TRAIN: 	 Epoch: 23 	 Loss: -0.005460185647409942
TRAIN: 	 Epoch: 23 	 Loss: -0.005385512879001908
TRAIN: 	 Epoch: 23 	 Loss: -0.005497144160067869
TRAIN: 	 Epoch: 23 	 Loss: -0.005558707553427666
TRAIN: 	 Epoch: 23 	 Loss: -0.005761537309312685
TRAIN: 	 Epoch: 23 	 Loss: -0.0057694357334791375
TRAIN: 	 Epoch: 23 	 Loss: -0.0058456221147655295
TRAIN: 	 Epoch: 23 	 Loss: -0.00585711606877989
TRA

100%|██████████| 947/947 [18:18<00:00,  1.16s/it]

Testing ....


ADE: 0.5252628102669415  FDE: 0.7080303378259996
**************************************************
******************************
Epoch: social-tag : 26
train_loss -0.006375991877505775
val_loss -0.0058540590740037415
{'min_val_epoch': 26, 'min_val_loss': -0.0058540590740037415}
******************************
TRAIN: 	 Epoch: 27 	 Loss: -0.006694732699543238
TRAIN: 	 Epoch: 27 	 Loss: -0.006911576492711902
TRAIN: 	 Epoch: 27 	 Loss: -0.0068857986479997635
TRAIN: 	 Epoch: 27 	 Loss: -0.007026535808108747
TRAIN: 	 Epoch: 27 	 Loss: -0.00699422862380743
TRAIN: 	 Epoch: 27 	 Loss: -0.006811045963938038
TRAIN: 	 Epoch: 27 	 Loss: -0.006286323303356767
TRAIN: 	 Epoch: 27 	 Loss: -0.00618249797844328
TRAIN: 	 Epoch: 27 	 Loss: -0.006247563727406992
TRAIN: 	 Epoch: 27 	 Loss: -0.006491138436831534
TRAIN: 	 Epoch: 27 	 Loss: -0.006644757570360194
TRAIN: 	 Epoch: 27 	 Loss: -0.006729720800649375
